<a href="https://colab.research.google.com/github/pklomkaew/HackHPC-HPCintheCity20/blob/gh-pages/fbi_cases_heatmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installation of necessary packages
pip install xlrd

In [83]:
pip install plotly==4.12.0

In [87]:
pip install plotly-geo

     |████████████████████████████████| 23.7MB 1.7MB/s 


In [84]:
pip install dash

In [135]:
# Importing packages
import plotly.figure_factory as ff 
import numpy as np
import pandas as pd
import requests

In [136]:
# Data Source: FBI
url2 = 'https://ucr.fbi.gov/crime-in-the-u.s/2018/crime-in-the-u.s.-2018/tables/table-8/table-8-state-cuts/georgia.xls/output.xls'
r2 = requests.get(url2)

with open('fbi_data.xls', 'wb') as f2:
    f2.write(r2.content)

In [137]:
# Data capentry
fbi_df = pd.read_excel('fbi_data.xls', header=4)

# Changing column names
fbi_df = fbi_df.rename(columns={"Population" : "population", "Violent\ncrime" : "violent_crime", "Murder and\nnonnegligent\nmanslaughter" : "murder", "Rape1" : "rape",})
fbi_df = fbi_df.rename(columns={ "Robbery" : "robbery", "Aggravated\nassault" : "assault", "Property\ncrime" : "property", "Burglary" : "burglary", "Larceny-\ntheft" : "larceny"})
fbi_df = fbi_df.rename(columns={ "Motor\nvehicle\ntheft" : "vehicle_theft", "Arson2" : "arson"})

fbi_df.head(5);

# Slicing dataframe
fbi_df = fbi_df[0:254]

# Upload coordinates and counties
ga_cities = pd.read_csv('ga-cities-coordinates.csv')
city = ga_cities.loc[:,['City']]
fbi_df = fbi_df.drop(['City'],axis=1)
fbi_df = pd.concat([city, fbi_df], axis=1, join='inner')


In [138]:
fbi_df = fbi_df.drop(['arson'], axis=1)

In [139]:
# Importing libraries
import plotly.express as px
import os

from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
ga_counties = {'type': 'FeatureCollection', 'features': []}

# Generate GA specific geojson
for county in counties['features']:
    if county['properties']['STATE'] in ['13']:
        ga_counties['features'].append(county)

ga_cities = pd.read_csv('ga-cities-coordinates.csv')
# coord_df = ga_cities.loc[:,['City','Latitude','Longitude']]
county_df = ga_cities.loc[:,['County']]

In [140]:
# Sum up cases of cities in the same counties
df = pd.concat([fbi_df, county_df], axis=1, join='inner')
case_by_county = df.groupby('County').sum()
case_by_county = case_by_county.set_index([pd.Index(range(0,128))])

In [177]:
# To get fips per county
gadata = pd.read_csv('Counties_GA_edited.csv')
gadata = gadata.rename(columns={'NAME10':'County', 'GEOID10':'fips', 'NAMELSAD10':'County_fullname'})

# Sort data by alphabetizing county 
gadata = gadata.sort_values(by=["County"])

# Isolating fips & combine fips with case_by_county
fips = gadata.loc[:,['County','fips','County_fullname']]
fips = fips.set_index([pd.Index(range(0,128))])
case_df = pd.concat([case_by_county, fips],axis=1)

case_df_perpop = case_df
case_df

,population,violent_crime,murder,rape,robbery,assault,property,burglary,larceny,vehicle_theft,County,fips,County_fullname
0,5013.0,73.0,1.0,3.0,8.0,61.0,388.0,67.0,293.0,28.0,Appling,13001,Appling County
1,1373.0,3.0,0.0,0.0,1.0,2.0,31.0,13.0,18.0,0.0,Atkinson,13003,Atkinson County
2,3486.0,25.0,1.0,0.0,3.0,21.0,198.0,65.0,123.0,10.0,Bacon,13005,Bacon County
3,593.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,Baker,13007,Baker County
4,18620.0,66.0,0.0,5.0,17.0,44.0,764.0,103.0,617.0,44.0,Baldwin,13009,Baldwin County
...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,4503.0,11.0,0.0,2.0,0.0,9.0,257.0,22.0,232.0,3.0,White,13311,White County
124,34704.0,93.0,0.0,11.0,21.0,61.0,1003.0,122.0,800.0,81.0,Whitfield,13313,Whitfield County
125,3876.0,1.0,0.0,0.0,1.0,0.0,49.0,13.0,36.0,0.0,Wilcox,13315,Wilcox County
126,4357.0,9.0,0.0,0.0,0.0,9.0,49.0,14.0,33.0,2.0,Wilkinson,13319,Wilkinson County


In [178]:
# Calculating the proportion of case by population
case_df_perpop['violent_crime'] = case_df_perpop['violent_crime']*100/case_df_perpop['population']
case_df_perpop = case_df_perpop.rename(columns={"violent_crime" : "Percent of Violent Crime by Population"})

# Data when mouse hovered
hover_data = ['County_fullname','Percent of Violent Crime by Population']

case_df

,population,violent_crime,murder,rape,robbery,assault,property,burglary,larceny,vehicle_theft,County,fips,County_fullname
0,5013.0,1.456214,1.0,3.0,8.0,61.0,388.0,67.0,293.0,28.0,Appling,13001,Appling County
1,1373.0,0.218500,0.0,0.0,1.0,2.0,31.0,13.0,18.0,0.0,Atkinson,13003,Atkinson County
2,3486.0,0.717154,1.0,0.0,3.0,21.0,198.0,65.0,123.0,10.0,Bacon,13005,Bacon County
3,593.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,Baker,13007,Baker County
4,18620.0,0.354458,0.0,5.0,17.0,44.0,764.0,103.0,617.0,44.0,Baldwin,13009,Baldwin County
...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,4503.0,0.244282,0.0,2.0,0.0,9.0,257.0,22.0,232.0,3.0,White,13311,White County
124,34704.0,0.267981,0.0,11.0,21.0,61.0,1003.0,122.0,800.0,81.0,Whitfield,13313,Whitfield County
125,3876.0,0.025800,0.0,0.0,1.0,0.0,49.0,13.0,36.0,0.0,Wilcox,13315,Wilcox County
126,4357.0,0.206564,0.0,0.0,0.0,9.0,49.0,14.0,33.0,2.0,Wilkinson,13319,Wilkinson County


# **Percent of Violent Crime by Population in Georgia by County in 2018**

In [180]:
# Generating Heatmaps
fig = px.choropleth(case_df_perpop,geojson=ga_counties, locations='fips',
                    color='Percent of Violent Crime by Population', color_continuous_scale="Viridis",
                    range_color=(0,1.5),scope = 'usa',hover_data=hover_data,
                    labels={'City':'Percent of Violent Crime by Population"'}, center = {"lat": 33.247875, "lon": -83.441162},
                    title='Percent of Violent Crime in Georgia Counties in 2018 by Population')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()